### RAG Pipeline complete from data ingestion to vector db


In [3]:
import os
from langchain_community.document_loaders import PyMuPDFLoader , PyPDFLoader , TextLoader , DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [7]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: air pollution act.pdf
  ✓ Loaded 43 pages

Processing: python_features.pdf
  ✓ Loaded 1 pages

Processing: python_use_cases.pdf
  ✓ Loaded 1 pages

Total documents loaded: 45


In [8]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [9]:
chunks = split_documents(all_pdf_documents)
chunks

Split 45 documents into 4 chunks

Example chunk:
Content: AIR (Prevention and control of Pollution) ACT...
Metadata: {'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'source': '..\\data\\pdf\\air pollution act.pdf', 'total_pages': 43, 'page': 0, 'page_label': '1', 'source_file': 'air pollution act.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'source': '..\\data\\pdf\\air pollution act.pdf', 'total_pages': 43, 'page': 0, 'page_label': '1', 'source_file': 'air pollution act.pdf', 'file_type': 'pdf'}, page_content='AIR (Prevention and control of Pollution) ACT'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'source': '..\\data\\pdf\\air pollution act.pdf', 'total_pages': 43, 'page': 42, 'page_label': '43', 'source_file': 'air pollution act.pdf', 'file_type': 'pdf'}, page_content='THANK YOU'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-22T14:02:32+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-12-22T14:02:32+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\python_features.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'python_f